# nn

> routines for neural network classes

In [ ]:
#| default_exp nn

In [ ]:
#| export
import torch
import torch.nn as nn

In [ ]:
class ScoreDynamicsDiffuser:
    def __init__(self, sigma_scale=1.0, schedule='linear', t_min=1e-3):
        self.schedule = schedule
        self.t_min = t_min
        self.sigma_scale = sigma_scale

        if schedule == 'linear':
            beta_0, beta_1 = 0.1, 20
            self.t_max     = 1
            self.gamma_t   = lambda t: torch.exp(-(beta_0*t/2+t.pow(2)/4*(beta_1-beta_0)))
            self.t_lambda  = lambda lmd: 2*torch.log(torch.exp(-2*lmd)+1)/(torch.sqrt(\
                beta_0**2 + (beta_1-beta_0)*2*torch.log(torch.exp(-2*lmd)+1)) + beta_0)
        else:
            raise NotImplementedError(f'Unknown noise schedule {schedule}')

        self.sigma_t    = lambda t: torch.sqrt(1-self.gamma_t(t)**2)
        self.lambda_t   = lambda t: torch.log(self.gamma_t(t)/self.sigma_t(t))
        self.lambda_min = self.lambda_t(torch.tensor(self.t_min)).item()
        self.lambda_max = self.lambda_t(torch.tensor(self.t_max)).item()


## Linear Noise Schedule [paper](https://arxiv.org/pdf/2011.13456.pdf) $$ (\alpha_{\tau}, \sigma_{\tau}) $$

### Need to look at the paper some more. Appendex B of paper has more info
$$ log \gamma_\tau= - \frac{\beta_1 - \beta_0}{2}\tau^{2} - \beta_0\tau$$
$$ \alpha_{\tau} = b \sqrt{\gamma_{\tau}} $$ 
$$ \sigma_{\tau} = \sqrt{1 - \gamma_{\tau}}$$ 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

/Users/pablo/software/miniconda3/envs/sd-playground/lib/python3.10/site-packages/nbdev/export.py:73: UserWarning: Notebook '/Users/pablo/software/sd-playground/nbs/02_nn.ipynb' uses `#|export` without `#|default_exp` cell.
Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.
  warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
